## Complete analysis of a numerical simulation
I need to improve each of the routine to plot things in iSALE

In [9]:
# common library
import numpy as np
from pathlib import Path
import sys
import os
import glob

# personal crater modelling library
import crater as cr # calculate geomorphology of impact crater
import ejecta as ej # calculate properties of ejected materials, errors when loading this module

### INPUT parameters

`method = 3` Take values equal to `2` or `3`, corresponds to the number of steps used after a tracer was ejected above a threshold above the surface <br>
`mode = 1` related to how the depth of the crater is calculated <br>
`poros = 1` Take values equal to `0, no porosity` or `1, porosity` <br>
`id_mat` 0 = everything, 2 only lower material, especially practical in layering modelling <br>
`transient` Take values equal to `0, max depth` or `1, max volume`, corresponds to how the transient crater is calculated <br>
`thresholdf` <br>
`g, surface gravity of the planetary body` <br>


In [3]:
method = 3
mode = 1
poros = 1  # if porous = 1, non porous = 0
id_mat = 0  # 0 = everything, 2 only lower material, especially practical in layering modelling
transient = 0 
thresholdf = 0.01
g = 1.62

#### path to folders containing jdata output files

In [11]:
# main directory which contains models with jdata
#path = '/var/tmp/mysshfs/stallo/collapse/ART3/results/'
pathm = '/home/nilscp/iSALE/Dellen/share/examples/'

folders =  ['demo2D']

In [15]:
# all functions should only take one model and it could be feed through loops

for f in folders:
    
    path = pathm + f + '/'

    os.chdir(path)

    # selection of folders
    #search = '*'
    models = ['demo2D']

    pathdata = '/home/nilscp/iSALE/results/tmp/data/'
    pathplots = '/home/nilscp/iSALE/results/tmp/plots/'

    ###############################################################################
    # 1. CALCULATE EXCAVATED CRATER DIMENSIONS
    ###############################################################################

    # Ve, de, De and shape of cavities 
    
    # (I should keep excavated from transient calculations...)
    #ej.main(path, folders, pathdata, method, thresholdf, g)


    ###############################################################################
    # 2. CALCULATE CRATER DIMENSIONS DURING CRATER EVOLUTION
    # 3. CALCULATE CRATER DIMENSIONS AT THE TRANSIENT CRATER
    # 4. CALCULATE CRATER DIMENSIONS AT THE FINAL CRATER (LAST FIVE TIMESTEPS)
    ###############################################################################

    cr.main(path, models, pathdata, mode, id_mat, transient)
    
    # I should also have plots about 

    ###############################################################################
    # 5. SAVE TXT FILE WITH X AND Y FOR THE TRANSIENT and FINAL CRATER
    ###############################################################################
    #cr.craterProfiles(models, path, pathdata, 0, 1)  # XY transient craters
    #cr.craterProfiles(models, path, pathdata, 0, 2)  # XY final craters
    # evo.XY_PROFILE(folders, path, pathdata, 19, 3) # arbitrary


###############################################################################
# 6. MAKE A FEW PLOTS IN ORDER TO HAVE THE POSSIBILITY TO DOUBLE CHECK SCRIPTS
# better to save plots in the same folder for all of them
###############################################################################
#plotting.main(pathdata, folders, pathplots)

###############################################################################
# 7. NOTE IN GOOGLE DRIVE THAT THE RESULTS ARE FREE OF ERRORS
###############################################################################

demo2D
Opened iSALE data file 'jdata.dat', with 201 time steps
Read in ['Den'] for timestep 0 (0.000e+00 s)
Read in ['Den'] for timestep 1 (1.258e-02 s)
Read in ['Den'] for timestep 2 (2.531e-02 s)
Read in ['Den'] for timestep 3 (3.862e-02 s)
Read in ['Den'] for timestep 4 (4.985e-02 s)
Read in ['Den'] for timestep 5 (6.266e-02 s)
Read in ['Den'] for timestep 6 (7.534e-02 s)
Read in ['Den'] for timestep 7 (8.795e-02 s)
Read in ['Den'] for timestep 8 (9.871e-02 s)
Read in ['Den'] for timestep 9 (1.113e-01 s)
Read in ['Den'] for timestep 10 (1.239e-01 s)
Read in ['Den'] for timestep 11 (1.365e-01 s)
Read in ['Den'] for timestep 12 (1.492e-01 s)
Read in ['Den'] for timestep 13 (1.602e-01 s)
Read in ['Den'] for timestep 14 (1.730e-01 s)
Read in ['Den'] for timestep 15 (1.860e-01 s)
Read in ['Den'] for timestep 16 (1.971e-01 s)
Read in ['Den'] for timestep 17 (2.104e-01 s)
Read in ['Den'] for timestep 18 (2.221e-01 s)
Read in ['Den'] for timestep 19 (2.341e-01 s)
Read in ['Den'] for timeste

In [ ]:
###################################################
# 2. EXTRACT INFORMATION FROM MODELS, READ PARAMETER
# AND COMPUTE SCALING LAWS (PI2, PIV, PID)
###################################################
nlayers = 1
paths = pathdata
for idx, modelname in enumerate(folders):
    pathi = path + modelname + '/INFO'
    readd.param(pathi, poros, paths + modelname +
                '/data/', nlayers)  # generate python_param
    # export.scalinglawsDRIM(paths + modelname + '/data/', modelname, nlayers) # generate scalinglaws DRIM
    export.scalinglaws(paths + modelname + '/transient/',
                       modelname, nlayers)  # generate scalinglaws


name = 'HOM_AUG'
search = 'C*'
# generate a table with L, pi2, pi3, piD ....
export.tablepi(paths, search, name, nlayers)
export.tablepiDRIM(paths, search, name, nlayers)
'''
***********************************************************************
'''
###################################################
# 3. READ PREVIOUSLY CREATED FILES
# AND COMPUTE SCALING LAWS
###################################################


filename = name + '_table.csv'
data = readd.pandas(filename)


'''
***********************************************************************
'''

###################################################
# 4. COMPUTE SCALING LAWS factor (beta, mu and K2)
# from 2.
###################################################

os.chdir(paths)

sea = 'C*'
lst = export.selecttargprop(paths, sea)

# Lmin = 50 # minimum prjectile length to take into account - important when models are not completely finished
#Lmax = 5000
Dsc = 16000
name = 'Call'

export.piDpiV(paths, lst)  # save pi2, piD in piD folder

# THIS IS ONLY FOR cohesionless materials (or materials with low cohesion)
# take previously generated file and calculate factors
export.savefactors(paths + 'piDpiV/', lst, name, Dsc)

'''
***********************************************************************
'''

###################################################
# 5. PLOT SCALING LAWS
###################################################

os.chdir(paths + 'piDpiV/')
lst = glob.glob('C*')
plott.piD(path, lst)  # save piD (first inspection check)

# rocks = data.loc[(output["por"]== 0.12) & (output["Friction_int"] == 1.0) & (output["Cohesion_int"] == 1e6)]

'''
***********************************************************************
'''

###################################################
# 6. GET TRANSIENT CRATERS
###################################################

os.chdir(path)
folders = glob.glob(search)  # or folders = ['CP20F']

trt = export.GET_TRANSIENT_TIME(folders, paths)
morph.SELECT_PROFILE(folders, trt, path, paths)

# plot transient craters
plott.transientPROFILE(paths, folders)


###################################################
# 6. GET RIM-RIM CRATERS
###################################################


path = '/var/tmp/mysshfs/stallo/layering/vdisc/resultsHOM/'
os.chdir(path)
folders = glob.glob('C*')
paths = '/media/nilscp/Ward/layering/vdisc/plots/P10/scalinglaws2/'
morph.SELECT_PROFILElast(folders, path, paths)
#SELECT_PROFILElast(folders, path, paths)
